In [1]:
import os
import torch
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda:0")

prompt = "no_prompt"
answer = "gas"
se_order = "as"
model_path = f"./output/GAS-Indo-{se_order}"

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
pipe = pipeline(task="text2text-generation",model=model, tokenizer=tokenizer, device=device)

In [4]:
import preprocess
data_path = "../data/absa/id/william/test.txt"
data_reader = preprocess.DataReader()
data_augmentator = preprocess.DataAugmentator()

In [5]:
model.config.max_length = 128

In [6]:
data_augmentator.task_example()

[{'se_order': 'aos', 'prompt': 'lego_absa', 'answer': 'lego_absa'},
 {'se_order': 'ao', 'prompt': 'lego_absa', 'answer': 'lego_absa'},
 {'se_order': 'as', 'prompt': 'lego_absa', 'answer': 'lego_absa'},
 {'se_order': 'a', 'prompt': 'lego_absa', 'answer': 'lego_absa'},
 {'se_order': 'o', 'prompt': 'lego_absa', 'answer': 'lego_absa'}]

In [7]:
data = data_reader.do(data_path)
augmented_data = data_augmentator.do(data, "aos", [{"se_order" : se_order, "prompt" : prompt, "answer" : answer}], 1, shuffle=False)

100%|██████████| 1000/1000 [00:00<00:00, 11228.83it/s]


In [8]:
import postprocess

answer_catcher = postprocess.AnswerCatcher()
cleaner = postprocess.Cleaner()

In [9]:
catch_answer_fn = getattr(answer_catcher, answer)

In [10]:
inputs = [el["input"] for el in augmented_data]
texts = [el for el in inputs]
targets = [catch_answer_fn(el["output"], el["se_order"], t) for el, t in zip(augmented_data, texts)]

In [11]:
preds = pipe(inputs, return_tensors=True)

In [12]:
preds = tokenizer.batch_decode([el["generated_token_ids"] for el in preds])

In [13]:
preds = cleaner.many(preds, remove=[tokenizer.pad_token, tokenizer.eos_token])

In [14]:
preds = [catch_answer_fn(p, el["se_order"], t) for p, el, t in zip(preds, augmented_data, texts)]

In [15]:
from evaluation import summary_score

score = summary_score(preds, targets)

In [16]:
score

{'recall': 0.7889512300388434,
 'precision': 0.8610456900612341,
 'f1_score': 0.8234234234234235}

In [17]:
preds

[[{'aspek': 'pelayanan', 'sentimen': 'positive'},
  {'aspek': 'kamar', 'sentimen': 'positive'},
  {'aspek': 'fasilitas', 'sentimen': 'positive'},
  {'aspek': 'airnya showernya', 'sentimen': 'negative'}],
 [{'aspek': 'NULL', 'sentimen': 'positive'}],
 [{'aspek': 'harga', 'sentimen': 'positive'},
  {'aspek': 'fasilitas', 'sentimen': 'positive'}],
 [{'aspek': 'kondisinya', 'sentimen': 'positive'},
  {'aspek': 'lift', 'sentimen': 'negative'}],
 [{'aspek': 'kamar', 'sentimen': 'positive'},
  {'aspek': 'bentuknya', 'sentimen': 'positive'},
  {'aspek': 'sinyal tv', 'sentimen': 'negative'}],
 [{'aspek': 'room', 'sentimen': 'positive'},
  {'aspek': 'linen', 'sentimen': 'negative'},
  {'aspek': 'kamar mandi', 'sentimen': 'negative'},
  {'aspek': 'handuk', 'sentimen': 'negative'},
  {'aspek': 'wifi', 'sentimen': 'negative'}],
 [{'aspek': 'air hangat', 'sentimen': 'negative'}],
 [{'aspek': 'hotelnya', 'sentimen': 'negative'}],
 [{'aspek': 'NULL', 'sentimen': 'positive'}],
 [{'aspek': 'bantal', 'se